In [1]:
import nest_asyncio
nest_asyncio.apply()
from pathlib import Path

import sys
sys.path.append("../")

from desci_sense.shared_functions.init import init_multi_chain_parser_config
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser
from desci_sense.evaluation.utils import get_dataset

from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)

In [2]:
config = init_multi_chain_parser_config(llm_type="google/gemma-7b-it",
                                        post_process_type="combined")

In [3]:
multi_chain_parser = MultiChainParser(config)

2024-05-16 12:50:24.144 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:66 - Initializing MultiChainParser. PostProcessType=combined
2024-05-16 12:50:24.154 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:73 - Initializing post parsers...
2024-05-16 12:50:24.155 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'multi_refs_tagger' 
2024-05-16 12:50:24.200 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'topics' 
2024-05-16 12:50:24.210 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'keywords' 
2024-05-16 12:50:24.219 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'hashtags' 


In [4]:
# example raw text posts
TEST_POST_TEXT_W_REF = """
I really liked this paper!
https://arxiv.org/abs/2402.04607
I really disagree with this paper!
https://mastodon.social/@UlrikeHahn@fediscience.org/111732713776994953
"""

TEST_POST_2_TEXT_W_REF = """
I really disagree with this paper!
https://arxiv.org/abs/2402.04607
"""

TEXT_POSTS = [TEST_POST_TEXT_W_REF, TEST_POST_2_TEXT_W_REF]

In [5]:
# convert raw text to RefPost format for input to the parser
inputs = [convert_text_to_ref_post(p) for p in TEXT_POSTS]

In [12]:
# or scrape posts by url
urls = [
        "https://mastodon.social/@psmaldino@qoto.org/111405098400404613",
        "https://mastodon.social/@UlrikeHahn@fediscience.org/111732713776994953",
        "https://mastodon.social/@ronent/111687038322549430",
    ]
posts = [scrape_post(url) for url in urls]

In [9]:
# batch process
results = multi_chain_parser.batch_process_ref_posts(inputs=inputs,active_list=["keywords", "topics"],batch_size=10)


2024-05-16 12:53:10.856 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-16 12:53:10.857 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://mastodon.social/redirect/statuses/111732713776994953
2024-05-16 12:53:10.871 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-16 12:53:10.872 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://arxiv.org/abs/2402.04607
2024-05-16 12:53:12.976 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:232 - Processing 2 posts with parsers: ['keywords', 'topics']
2024-05-16 12:53:12.979 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:234 - Instantiating prompts...


  0%|          | 0/4 [00:00<?, ?it/s]

2024-05-16 12:53:13.029 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:248 - Invoking parallel chain...
2024-05-16 12:53:16.783 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:259 - Post processing 2 results...
2024-05-16 12:53:16.786 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:271 - Done!


In [6]:
#
url = "https://twitter.com/mbauwens/status/1779543397528740338"
post = scrape_post(url)

In [7]:
# single process
result = multi_chain_parser.process_ref_post(post)

2024-04-30 15:42:45.141 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://icandecide.org/v-safe-data/
2024-04-30 15:42:45.164 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://icandecide.org/pfizer-documents/
2024-04-30 15:43:04.309 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:184 - Processing post with parsers: ['refs_tagger', 'topics', 'keywords', 'hashtags']
2024-04-30 15:43:04.310 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:186 - Instantiating prompts...
2024-04-30 15:43:04.311 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:191 - Invoking parallel chain...


In [8]:
result.topics

['sustainability',
 'software & hardware',
 'academia',
 'climate',
 'news',
 'health',
 'technology',
 'research']

In [16]:
result.filter_classification

<SciFilterClassfication.RESEARCH: 'research'>

In [43]:
results[0].reference_tagger

['review', 'disagrees', 'agrees']

In [19]:
result.topics

['economics', 'research']

In [16]:
l = ["Topics: "+x.debug['topics']['full_text']+"Keywords: "+x.debug['keywords']['raw_text'] for x in results]
print(l)

['Keywords: **Reasoning Steps:**\n\n1. **Identify the main topic of the post:** The post discusses the potential for manipulation of citations on Google Scholar. Therefore, the main topic is "scientific integrity."\n\n\n2. **Identify the specific topics discussed in the post:** The post also discusses self-citations and citation cartels. These topics are related to the main topic.\n\n\n3. **Consider the context of the post:** The post is a personal opinion piece, and the author is expressing their disagreement with a paper. Therefore, the topic of "disagreement" is also relevant.\n\n\n**Candidate Topics:**\n\n- Scientific integrity\n- Self-citations\n- Citation cartels\n- Google Scholar\n- Scientific evaluation\n- Disagreement\n- Academic fraud\n\n**Final Answer:**\n\n**Final Topics:**\n\n- Scientific integrity\n- Self-citations\n- Citation cartels\n- Google Scholar\n- Scientific evaluation\n- Disagreement \n\n ##Allowed terms: [\'technology\', \'science\', \'academia\', \'research\', 

In [10]:
print(results[0].debub)

research_keyword='not-academic' filter_classification=<SciFilterClassfication.RESEARCH: 'research'> item_types=['preprint', 'webpage'] reference_urls=['https://arxiv.org/abs/2402.04607', 'https://mastodon.social/redirect/statuses/111732713776994953'] reference_tagger=None multi_reference_tagger=None keywords=['CitationAnalysis', 'GoogleScholar', 'MachineLearning', 'NetworkAnalysis', 'CitationFraud'] topics=['art'] hashtags=[] metadata_list=[RefMetadata(citoid_url='http://arxiv.org/abs/2402.04607', url='https://arxiv.org/abs/2402.04607', item_type='preprint', title='Google Scholar is manipulatable', summary="Citations are widely considered in scientists' evaluation. As such, scientists may be incentivized to inflate their citation counts. While previous literature has examined self-citations and citation cartels, it remains unclear whether scientists can purchase citations. Here, we compile a dataset of ~1.6 million profiles on Google Scholar to examine instances of citation fraud on th

In [14]:
import wandb
import pandas as pd

wandb.login()

api = wandb.Api()

#TODO move from testing
run = wandb.init(project="testing", job_type="evaluation")

# get artifact path

dataset_artifact_id = (
        'common-sense-makers/filter_evaluation/labeled_tweets_no_threads:v1'
    )

# set artifact as input artifact
dataset_artifact = run.use_artifact(dataset_artifact_id)

# initialize table path
# add the option to call table_path =  arguments.get('--dataset')

# download path to table
a_path = dataset_artifact.download()
print("The path is",a_path)

# get dataset file name

table_path = Path(f"{a_path}/labeled_data_table_no_threads.table.json")


# return the pd df from the table
#remember to remove the head TODO
df = get_dataset(table_path)

table_path = Path(f"{a_path}/handles_chart.table.json")

df_handles = get_dataset(table_path)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shahar-r-oriel (common-sense-makers). Use `wandb login --relogin` to force relogin


wandb:   2 of 2 files downloaded.  


The path is /Users/shaharorielkagan/sensemakers/nlp/notebooks/artifacts/labeled_tweets_no_threads:v1


In [15]:
import concurrent.futures
from tqdm import tqdm

def process_text(text):
    return convert_text_to_ref_post(text)

# Assuming df['Text'] is your DataFrame column
with concurrent.futures.ThreadPoolExecutor() as executor:
    inputs = list(tqdm(executor.map(process_text, df['Text']), total=len(df['Text'])))


100%|██████████| 467/467 [00:49<00:00,  9.41it/s]


In [16]:
inputs = inputs[:10]

In [31]:
# batch process
results = multi_chain_parser.batch_process_ref_posts(inputs,active_list=['multi_refs_tagger'],batch_size=10)

2024-05-06 15:52:15.507 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-06 15:52:15.512 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://civilizationemerging.com/
2024-05-06 15:52:15.525 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-06 15:52:15.526 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://en.wikipedia.org/wiki/Epic_of_evolution
2024-05-06 15:52:15.527 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-05-06 15:52:15.528 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://wiki.p2pfoundation.net/Category:Peerproduction#With_the_advent_of_the_P2P_Mode_of_Production%2C_the_community_and_its_common_is_now_the_appropriate_

  0%|          | 0/10 [00:00<?, ?it/s]

2024-05-06 15:52:22.695 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:245 - Invoking parallel chain...
2024-05-06 15:52:27.198 | WARNING  | desci_sense.shared_functions.postprocessing.output_processors:parse_result:360 - Failed to parse result: Invalid json output: [System error]: ```python

sub_answer = SubAnswer()


sub_answer.reasoning_steps = "The post describes a quote of the day that talks about the multiplicity of ownership designs and the need to ecologize finance. The post also discusses the importance of redesigning social architectures and making finance more inclusive. The post references a category page on the P2P Foundation website. The post quotes text from the referenced page. Therefore, the tags that best characterize the post's relation to the reference are 'quote' and 'reference'."


sub_answer.candidate_tags = ["quote", "reference", "ecologize", "inclusive", "social architectures"]


sub_answer.final_answer = ["quote", 

In [19]:
df=df[:10]

In [30]:
df["Predicted Labels"] = [x.multi_reference_tagger for x in results]
df["urls"] = [x.reference_urls for x in results]



/var/folders/j3/1_zy_b4s2517g8j_rjjdhd4h0000gn/T/ipykernel_4079/1770973723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Predicted Labels"] = [x.multi_reference_tagger for x in results]
/var/folders/j3/1_zy_b4s2517g8j_rjjdhd4h0000gn/T/ipykernel_4079/1770973723.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["urls"] = [x.reference_urls for x in results]


[['https://en.wikipedia.org/wiki/Epic_of_evolution', 'https://en.wikipedia.org/wiki/Epic_of_evolution'], ['https://wiki.p2pfoundation.net/Category:Thermodynamic_Efficiencies'], ['https://lifehacker.com/tech/ai-is-running-out-of-internet'], ['https://www.journaloffreespeechlaw.org/'], ['https://wiki.p2pfoundation.net/Andrew_Targowski%27s_Classification_of_the_Civilizational_Approaches_To_Human_History'], ['https://civilizationemerging.com/', 'http://emergentcivilization.com/'], ['https://integralpermaculture.wordpress.com/peter-pogany/', 'https://integralpermaculture.wordpress.com/peter-pogany/'], ['https://wiki.p2pfoundation.net/Category:Peerproduction#With_the_advent_of_the_P2P_Mode_of_Production,_the_community_and_its_common_is_now_the_appropriate_scale'], ['https://wiki.p2pfoundation.net/Category:Peerproperty#Short_Citations'], ['https://autocatallaxy.com/']]
+++++
[['https://en.wikipedia.org/wiki/Epic_of_evolution', 'https://en.wikipedia.org/wiki/Epic_of_evolution'], ['https://wiki